# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import ollama
import gradio as gr

In [2]:
MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'
MODEL_ANTHROPIC = 'claude-3-haiku-20240307'
MODEL_GEMINI = 'gemini-2.0-flash-exp'

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyDZ


In [4]:
openai = OpenAI()

claude = anthropic.Anthropic()

google.generativeai.configure()

In [5]:
#Prompt del sistema
system_prompt = "Eres un experto en LLM, inteligencia artificial y python \
el cual se le van a proporcionar una o varias preguntas técnicas relacionadas a los temas a los que eres experto \
y tienes que responder de tal manera que un estudiante que es muy novato y apenas está aprendiendo entienda"
system_prompt += "Siempre se preciso. si no sabes la respuesto, dilo."

In [6]:
# here is the question; type over this to ask something new
question = """
Por favor explica que hace este código y porque:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [7]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL_GPT, messages=messages)
    return response.choices[0].message.content
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [8]:

import base64
from io import BytesIO
from PIL import Image

In [9]:

def artist(city):
    image_response = openai.images.generate(
            model="dall-e-3",
            prompt=f"",
            size="1024x1024",
            n=1,
            response_format="b64_json",
        )
    image_base64 = image_response.data[0].b64_json
    image_data = base64.b64decode(image_base64)
    return Image.open(BytesIO(image_data))


In [10]:
#image = artist("New York City")
#display(image)

In [11]:
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time

def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # Student Dominic found that this was needed. You could also try commenting out to see if not needed on your PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

talker("Hola")

In [12]:
def response_gpt(history):
    messages = [{"role": "system", "content": system_prompt}] + history
    response = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=messages,
        #stream=True
    )
    result = response.choices[0].message.content
    return result
    #for chunk in stream:
        #result += chunk.choices[0].delta.content or ""
        #yield result

In [13]:
def response_claude(history):

    cleaned_history = [
    {"role": msg["role"], "content": msg["content"]}
    for msg in history
    ]
    result = claude.messages.create(
        model=MODEL_ANTHROPIC,
        max_tokens=1000,
        temperature=0.7,
        system=system_prompt,
        messages=cleaned_history,
    )

    response = result.content[0].text
    return response
    #response = ""
    #with result as stream:
        #for text in stream.text_stream:
            #response += text or ""
            #yield response

In [14]:
def response_llama(history):
    messages = [{"role":"system", "content": system_prompt}] + history
    result = ollama.chat(
        model = MODEL_LLAMA,
        messages = messages,
        #stream = True
    )
    return result['message']['content']
    #display_handle = display(Markdown(""), display_id=True)
    #for chunk in stream:
        #response += chunk['message']['content'] or ''
        #response = response.replace("```", "").replace("markdown","")
        #update_display(Markdown(response), display_id=display_handle.display_id)

In [15]:
def response_model(history, model):
    result = ''
    if model=="GPT":
        result = response_gpt(history)
    elif model=="Claude":
        result = response_claude(history)
    elif model=="Llama":
        result = response_llama(history)
    else:
        raise ValueError("Unknown model")
    return result
    #yield from result

In [16]:
def chat(history, model):
    image = None
        
    reply = response_model(history, model)
    history += [{"role":"assistant", "content":reply}]

    # Comment out or delete the next line if you'd rather skip Audio for now..
    talker(reply)
    
    return history, image

In [17]:
with gr.Blocks() as ui:
    with gr.Row():
        chatbot = gr.Chatbot(height=500, type="messages")
        image_output = gr.Image(height=500)
    with gr.Row():
        entry = gr.Textbox(label="Chat with our AI Assistant:" , scale=3)
        model = gr.Dropdown(choices = ["GPT", "Claude", "Llama"], label="Select model", scale=1)
    with gr.Row():
        clear = gr.Button("Clear")

    def do_entry(message, history, selection):
        history += [{"role": "user", "content": f"{message} "}]
        return "", history, selection

    entry.submit(do_entry, inputs=[entry, chatbot, model], outputs=[entry, chatbot, model]).then(
        chat, inputs=[chatbot, model], outputs=[chatbot, image_output]
    )
    clear.click(lambda: None, inputs=None, outputs=chatbot, queue=False)

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.
